In [2]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [3]:
import numpy as np
import pandas as pd

In [4]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [5]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [6]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [7]:
# reflect an existing database into a new model
base = automap_base()

# reflect the tables
base.prepare(engine, reflect=True)

In [8]:
# We can view all of the classes that automap found
base.classes.keys()

['measurement', 'station']

In [9]:
# Save references to each table
stat = base.classes.station
meas = base.classes.measurement


In [30]:
# Create our session (link) from Python to the DB
sess = Session(engine)


# Exploratory Climate Analysis

In [31]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results

# Calculate the last data point in db
last = sess.query(meas.date).order_by(meas.date.desc()).first()
last

('2017-08-23')

In [32]:
# Calculate the date 1 year ago from the last data point in the database
one_year = dt.date(2017,8,23) - dt.timedelta(days=365)
print(one_year)

2016-08-23


In [33]:
# Perform a query to retrieve the data and precipitation scores
precip = sess.query(meas.date, meas.prcp)\
                    .filter(meas.date <= '2017-08-23')\
                    .filter(meas.date >= '2016-08-23').all()

# Save the query results as a Pandas DataFrame 
precip_df = pd.DataFrame(data=precip, columns=['date', 'precipitation'])

# Set the index to the date column
precip_df = precip_df.set_index('date')

# Sort the DataFrame by date
sorted_df = precip_df.sort_index()

In [86]:
# Use Pandas Plotting with Matplotlib to plot the data
bar = sorted_df.plot(kind='line', figsize = (12,6), rot=90)

# Set X and Y labels
plt.xlabel('Date')
plt.ylabel('Inches')

# Set Title and Legend
plt.title('Precipitation from 2016-08-23 to 2017-08-23')
plt.legend(['Precipitation'])

# Display Graph
plt.tight_layout()
plt.show()

# Save Image
bar.figure.savefig('Images/precipitation_bar.png')


In [47]:
# Use Pandas to calcualte the summary statistics for the precipitation data
summary = precip_df.describe()
summary

,precipitation
count,2021.000000
mean,0.177279
std,0.461190
min,0.000000
25%,0.000000
50%,0.020000
75%,0.130000
max,6.700000


In [54]:
# Design a query to show how many stations are available in this dataset?
stations = sess.query(meas).group_by(meas.station).count()
stations

9

In [78]:
# What are the most active stations? (i.e. what stations have the most rows)?
active = sess.query(meas.station, stat.name, func.count(meas.station))\
                    .filter(meas.station == stat.station)\
                    .group_by(meas.station)\
                    .order_by(func.count(meas.station).desc())\
                    .all()

# List the stations and the counts in descending order.                    
active


[('USC00519281', 'WAIHEE 837.5, HI US', 2772),
 ('USC00519397', 'WAIKIKI 717.2, HI US', 2724),
 ('USC00513117', 'KANEOHE 838.1, HI US', 2709),
 ('USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 2669),
 ('USC00516128', 'MANOA LYON ARBO 785.2, HI US', 2612),
 ('USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 2202),
 ('USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', 1979),
 ('USC00517948', 'PEARL CITY, HI US', 1372),
 ('USC00518838', 'UPPER WAHIAWA 874.3, HI US', 511)]

In [82]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station?
active_summary = sess.query(meas.station, func.min(meas.tobs), func.max(meas.tobs),
                                func.avg(meas.tobs))\
                            .filter(meas.station == 'USC00519281')\
                            .all()
active_summary


[('USC00519281', 54.0, 85.0, 71.66378066378067)]

In [79]:
# Choose the station with the highest number of temperature observations.
highest = sess.query(meas.station, func.count(meas.station))\
                    .group_by(meas.station)\
                    .order_by(func.count(meas.station).desc())\
                    .first()
highest



('USC00519281', 2772)

In [90]:
# Query the last 12 months of temperature observation data for this station
temp_obs = sess.query(meas.tobs)\
                     .filter(meas.date >= '2016-08-23')\
                     .filter(meas.date <= '2017-08-23')\
                     .filter(meas.station == 'USC00519281')\
                     .all()

# Load the data into a DataFrame
temp_obs_df = pd.DataFrame(temp_obs)

In [106]:
# Plot the results as a histogram
histo = temp_obs_df.plot.hist(bins=12, figsize=(12,8))

# Set X and Y labels
plt.xlabel('Temperature')
plt.ylabel('Frequency')

# Set Title and Legend
plt.title('Temperature Observations: Station USC00519281\n(2016-08-23 to 2017-08-23))')
plt.legend(['TObs'])

# Display Graph
plt.tight_layout()
plt.show()

# Save Image
histo.figure.savefig('Images/tobs_histogram.png')

## Bonus Challenge Assignment

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
# def calc_temps(start_date, end_date):
#    """TMIN, TAVG, and TMAX for a list of dates.
    
#    Args:
#        start_date (string): A date string in the format %Y-%m-%d
#        end_date (string): A date string in the format %Y-%m-%d
        
#    Returns:
#        TMIN, TAVE, and TMAX
#    """
    
#    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
#        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
# print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.


In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)


In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation



In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

# def daily_normals(date):
#    """Daily Normals.
    
#    Args:
#        date (str): A date string in the format '%m-%d'
        
#    Returns:
#        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
#    """
    
#    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
#    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
# daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index
